# Theoretical points -- SQL

source:  Complete SQL Mastery -- instructor: Mosh Hamedani 

**0. Connecting to a MySQL Database with Python (Pandas) and Executing pure SQL Queries**

In order to run pure SQL code in python, I followed the following procedure (this way allows me to push all my codes, which were run in jupyter notebook, into my GitHub repo rather than running SQL queries in my local using MySQL Workbench as a database management tool to interact with MySQL):

+ Create a connection to my MySQL server:

        def connector (database, password=password, host='localhost', 
                           user='root', charset='utf8mb4',):

                con = pymysql.connect(
                                    host=host,
                                    user=user,
                                    password=password,
                                    charset=charset, 
                                    database=database)

                return con
            
        con = connector (database) 
        
+ Execute the SQL query

        cursor = con.cursor()
        cursor.execute("""
        
        SELECT *
        FROM orders
        
        """)
        
        con.commit()
        
        cursor.close()
        con.close()
        
        results = cursor.fetchall()
        
        df = pd.DataFrame(results, columns=[desc[0] for desc in cursor.description])
 

some notes on **cursor object**:

In Python, we can interact with MySQL databases using the MySQL Connector/Python module, which provides a Cursor object for executing SQL queries.

A cursor is an object that allows me to execute SQL statements and retrieve data from a MySQL database. It acts as a pointer to a specific row or set of rows within a result set. The Cursor object is created by calling the cursor() method on a MySQL connection object.

Once the cursor object is created, I can execute SQL queries using the execute() method. The results of the query can be retrieved using the fetchall() method.

It's important to note that after executing a SQL query, you need to either fetch all the rows returned by the query or use one of the other fetch methods (fetchone(), fetchmany()) to retrieve specific rows. **After you're done working with the cursor object, you should close it using the close() method to free up system resources.**

**note on pd.read_sql vs. using cursor object**:

If I just need to execute a simple SELECT statement and retrieve the results as a Pandas DataFrame, then using the pd.read_sql() function is a more convenient option than using a cursor object.

But if I need to execute more complex SQL queries that involve multiple statements, transaction management, or other advanced features, then I may need to use a cursor object instead of pd.read_sql().


**1. What is a Database?**

A database is a collection of data stored in a format that can be easily accessed. In order to manage our databases we use a software called **database management system (DBMS)**. DBMS are categorized as two broad categories:

+ Relational

+ NoSQL

**2. Relational DBMS**

We store data in tables that are linked to each other using relationships, each table stores data about the specific type of object. We use **Structured Query Language (SQL)** to work with these relational DBMS. We use SQL to query or modify our data. 
There are various RDBMS like:

+ MySQL
+ SQL Server
+ Oracle
+ ...

All of these flavors are based on the standard specifications of SQL. So what I learn in this course mostly can be applied to any of these RDBMS. In this course we use MySQL, which is the most popular open source database in the world. 


**3. Non relational databases**

In Non relational databases we don’t have tables or relationships, these databases are very different from relational databases and NoSQL systems don’t understand SQL and they have their own query language. 

**4. Course structure**

What you will learn: 

+ Retrieving data
+ Inserting data
+ Updating data
+ Deleting data

Even though we use MySQL in this course, what you will be learning will be applicable at least 90% to other DBMSs. There might be a slight difference in syntax though. 

You also learn

+ Summarizing data for creating reports
+ Writing complex queries using subqueries
+ Built-in functions 

As you process you see you are writing the same queries again and again and so you will learn about the followings for storing queries and reusing them later:

+ Views
+ Stored procedures 

These are great for increasing your productivity.

Then we move on to advanced topics:

+ Triggers
+ Events
+ Transactions
+ Concurrency

If you want to stand out among the crowds you have to learn about:

+ Designing databases

Mosh dedicated a complete in-depth section on it. 

Then we look at 
+ Indexing for high performance, indexes are essential when your database grows in size, we can speed up our queries using indexes. If you have billions of records you want to query pretty quick using indexes 

Finally we finish the course by 
+ Securing databases

## Retrieving Data From a Single Table

**5. The SELECT statement**

    USE sql_store; 
    
    SELECT * (in front of that we specify the columns that we want to retrieve, * means all columns) 
    FROM customers (here we specify the table that we want to query)
    WHERE customer_id = 1 (we use this clause to filter data)
    ORDER BY first_name (we use this clause to sort data and here we specify the columns that we want to sort the results on)


The first step to write a query to get data from a database is to select the database, the query that we write will be executed against that database. We use **USE** keyword to select the database. 

Points:
+ SQL is NOT a case sensitive language meaning we can use USE sql_store or use sql_store BUT as the best practice we should **capitalize all the SQL keywords and everything else in lower case**
+ When you have multiple SQL statements we need to terminate each statement using a semicolon (;) 
+ Comment in SQL is two hyphens –- and the SQL engine will not execute these comments 

After executing:
+ USE sql_store (by double clicking on the sql_store we can achieve the same thing)

The sql_store is marked as bold in the schemas database (in the left panel) 

**The order of these clauses (SELECT FROM WHERE ORDER BY) matter. And if we don’t follow the above order we get a syntax error.

The three clauses of **FROM, WHERE, ORDER BY are all optional** but we mostly use them. 
Also we don’t need to put different clauses in different lines b/c tab, whitespaces, and line breaks are ignored when SQL code is executed. So we may have

    SELECT * FROM customers WHERE customer_id = 1 ORDER BY first_name 

This is fine for simple queries but as your queries get more complex it is better to put each clause on a new line. 

**6. The AND, OR, and NOT operators**

AND is operated first! But we use parenthesis to make our code cleaner and easier to understand. 

**7. The IN operator in SQL**

        USE sql_stores; 
        
        SELECT *
        FROM customers 
        WHERE state = ‘VA’ OR state = ‘FL’ OR state = ‘GA’ 
        
But we can use IN operator to have a cleaner code like

        SELECT *
        FROM customers 
        WHERE state IN (‘VA’, ‘GA’, ‘FL’) -- The order does NOT matter 

We also can use NOT operator like to get the customers outside of these states:

        SELECT *
        FROM customers 
        WHERE state NOT IN (‘VA’, ‘GA’, ‘FL’)
        
**8. The BETWEEN operator in SQL**

Whenever we are comparing an attribute with a range of values we can use **BETWEEN operator** to make you code shorter and cleaner like:

    USE sql_store;
    SELECT *
    FROM customers 
    WHERE points BETWEEN 1000 AND 3000 -- The range values is inclusive. 
    
**9. The LIKE operator**

How to retrieve rows that match a specific string pattern? Like we want to get the customers whose last names starts with “b”:

**Use % to indicate any number of characters after “b”, also it doesn’t matter if it is a lower case or upper case “b”**


    SELECT *
    FROM customers
    WHERE last_name LIKE “b%”
    
The percent sign doesn’t have to be at the end but it could be anywhere like we want to get customers with “b” character in their last name whether in the beginning or at the end or in the middle, for that we change our pattern to **“%b%”** meaning we can have any number of characters before or after “b”:

    SELECT *
    FROM customers
    WHERE last_name LIKE “%b%”
    
For the customers with last name ending in “y” we use this pattern “%y”. We also have **underscore “_”that matches a single character** like in the following we will get the customers whose last names are exactly TWO characters long and it ends in y and we don’t care what the first character is:

    SELECT *
    FROM customers 
    WHERE last_name LIKE “_y”
    
Another pattern is “b____y” and you know what it means😊 

The LIKE operator in MySQL is an older operator and we have a newer one which is more powerful which allows us to search for any string pattern. We will discuss next. 

**10. The REGEXP operator**

REGEXP is short for regular expression. REGEXP is extremely powerful when it comes to **searching for strings**. They allow us to search for more complex patterns. **The newer and better operator compared to LIKE.**

Some examples:

Let’s say if we want to have customers whose last names include field, if we want to use LIKE operator:

    SELECT *
    FROM customers
    WHERE last_name LIKE “%field%”
    
BUT if we use REGEXP we do NOT need % to indicate that the field word could be anywhere and the above query would be simplified to as:

    SELECT *
    FROM customers
    WHERE last_name REGEXP “field”
    
In REGEXP we have additional characters that we don’t have in LIKE operator. Like

+ ^	to indicate the beginning of the string like

        WHERE last_name REGEXP “^field”	means the last_name MUST start with field
        
+ $	to indicate the end of the string like 

        WHERE last_name REGEXP “field$” means the last name must end with field
        
+ We can also search for multiple words, to do so we use **pipe (|) for logical or** for multiple search patterns, like to get customers who have field or mac in their last names:

        WHERE last_name REGEXP “field|mac” 
        
+ We can take this to the next level like finding customers who have the words field or mac or rose in their last names:

        WHERE last_name REGEXP “fiels|mac|rose”

        WHERE last_name REGEXP “^field|mac$|rose”
        
+ We want to get customers who have letter e in their last names and also before the letter e there is either g or i, to achieve this we use square brackets [] and inside it we can add multiple characters like “[gi]e”

        WHERE last_name REGEXP “[gi]e”
        
        WHERE last_name REGEXP “e[gi]”
        
+ We can also supply a **range of characters** like if we want to have any character from a to h before the letter e

        WHERE last_name REGEXP “[a-h]e”
        
There are more special characters in MySQL than listed above but honestly the ones on above are the ones you use 90 % of the times. So just learn these and you are good to go! 

**11. The IS NULL operator**

How to look for records that miss an attribute? Like if we want to find all the customers that don’t have a phone number and like we want to send an email to them and say please provide phone number. To do so we use IS NULL operator:

    SELECT *
    FROM customers
    WHERE phone IS NULL 
    
Here we also can use NOT operator to get the customers who do have a phone like

    SELECT *
    FROM customers
    WHERE phone IS NOT NULL

**12. The ORDER BY clause**

How to sort data in your SQL queries? 

By default the data in our query is sorted by the customer_id. Why is the customer_id column the default sort column? 

Because the customer_id column is the **primary key column** for the table customers. You can figure it out by clicking the tools like button on the right of the table customers name and then see the customer table in the design mode. Here you see that there is a yellow key just before the customer_id meaning this is a primary key column for this table. **In relational databased every table should have a primary key column and the values in that column should uniquely identify the records in that table**. So that is why in our query the data is sorted by the custiomer_id which is the primary key. To sort the customers by different column:

    SELECT *
    FROM customers 
    ORDER BY first_name 
    
To reverse the order:

    SELECT *
    FROM customers
    ORDER BY first_name DESC
    
We can also sort data by multiple columns for example:

Let’s say we want to first sort customers based on their state and then within each state we want to sort them by their first name:

    SELECT *
    FROM customers
    ORDER BY state, first_name 
    
We can also use a descending argument anywhere like:

    SELECT *
    FROM customers
    ORDER BY state, first_name DESC 
    
The difference between MySQL and other DBMSs is that in MySQL we can sort by any columns whether that column is in the SELECT clause or NOT like:

    SELECT first_name, last_name
    FROM customers
    ORDER BY state
    
Here although I don’t have state column in SELECT clause but I can sort data based on that. But other DBMSs sometimes yell at you when you try to do sorting like this. 

We also can sort data by an alias like:

    SELECT first_name, last_name, 10 AS points 
    FROM customers
    ORDER BY points, first_name 
    
AVOID: you can also sort data by columns position like:

    SELECT first_name, last_name
    FROM customers
    ORDER BY 1, 2 – these are the orders of the columns in the SELECT clause 
    
This query sorts data by first_name and then last_name.

**BUT avoid** this b/c if in the future you have a change in the columns in the SELECT clause like:

    SELECT birth_data, first_name, last_name 
    FROM customers
    ORDER BY 1, 2
    
The data would be sorted differently and generates unexpected results and so **avoid sorting by column’s positions INSTEAD ALWAYS SORT BY COLUMNS’NAMES.**

**13. The LIMIT clause**

How to limit the number of records from your query? Like if we only want to get the three first customers we use the LIMIT clause:

    SELECT *
    FROM customers
    LIMIT 3
    
We can optionally supply an **offset which is useful for situations we want to paginate data.** Like:

    SELECT *
    FROM customers
    LIMIT 6, 3 
    
**Here 6 is an offset and tells MySQL to skip the first 6 records and then pick three records**.

Exercise: get the top three loyal customers, like the ones with highest points 

    SELECT *
    FROM customers
    ORDER BY points DESC 
    LIMIT 3 

**POINT: Again the order matters! The LIMIT clause should always come at the end.** 

# Retrieving Data From Multiple  Tables

**14. Inner Joins**

So far we only selected columns from a single table but in the real world we quite often select columns from multiple tables. Like:  

    SELECT *
    FROM orders
    JOIN customers 
        ON orders.customer_id = customers.customer_id 
    
this JOIN is the same as INNER JOIN where INNER is optional, we also have OUTER JOIN, which will be discussed later
    
With this query we tell MySQL that hey whenever you are joining the orders table with customers table make sure that the customer_id column in the orders table equals customer_id column in the customers table.  

The output would be all the columns from the two tables, first all the columns from orders table, b/c we have listed this table first, and then all the columns from the customers table. Let’s simplify the results:

    SELECT order_id, first_name, last_name
    FROM orders
    JOIN customers 
        ON orders.customer_id = customers.customer_id 
    
What if we want to display the customer_id as well? 

    SELECT order_id, customer_id, first_name, last_name
    FROM orders
    JOIN customers 
        ON orders.customer_id = customers.customer_id 
    
This query throws an error b/c I have the column customer_id in two tables! The error says that customer_id column in the field list is ambiguous! So I need to specify from which table like:

    SELECT order_id, orders.customer_id, first_name, last_name
    FROM orders
    JOIN customers 
        ON orders.customer_id = customers.customer_id         
Now it works! 

Points: 

+ In situations where we have same column in multiple tables we need to prefix the name of the column with the name of the table like above

+ ON phrase specifies the basis based on which we want to join the tables, after ON we need to specify the condition 

+ We can use alias to make our code cleaner, NOTE: **if you give an alias to a table you have to use that alias everywhere else otherwise you get an error.** 

        SELECT order_id, o.customer_id, first_name, last_name
        FROM orders o
        JOIN customers c 
            ON o.customer_id = c.customer_id 

**15. Joining across databases**

How to combine columns from tables across multiple databases?

What we did so far was to combine columns from multiple tables within a database, sql_store, but now we want to combine columns from tables across multiple databases. We need to prefix the table in JOIN with its database name:

    USE sql_store;
    SELECT *
    FROM order_items oi
    JOIN sql_inventory.products p 
        ON oi.product_id = p.product_id 
    
So we successfully joined tables across multiple databases.

We need this **prefix, sql_inventory.products, b/c the database we write this query against is the sql_store database**. Also 

    USE sql_inventory;
    SELECT *
    FROM sql_store.order_items oi 
    JOIN products p 
        ON oi.product_id = p.product_id 
        
**LESSON** ----> **I only have to prefix the tables that are NOT part of the current database. And so my query would be different depending on the current database.**

**16. Self Joins**

In SQL we can also join a table with itself. Let’s write a query to get each employee and their manager 

    USE sql_hr;
    SELECT 
        e.employee_id, 
        e.first_name, 
        m.first_name AS Manager
    FROM employees e
    JOIN employees m 
        ON e.reports_to = m.employee_id
        
Joining a table with itself is pretty similar to joining a table with another table the only difference is that **we have to use different aliases and we have to prefix each column with an alias**, this is called self join. 

**17. Joining Multiple Tables**

How to join more than TWO tables when writing a query? 

We want to write a query to join the orders table with two tables: the customers table and the order_statuses table. 

    USE sql_store;
    SELECT 
        o.order_id, 
        o.order_date, 
        c.first_name, 
        c.last_name, 
        os.name AS status 
    FROM orders o
    JOIN customers c
        ON o.customer_id = c.customer_id 
    JOIN order_statuses os 
        ON o.status = os.order_status_id 
        
Exercise: write a query to join the payment table with the payment methods table as well as the client table and produce a report that shows the payment with more details such as the client’s name and the payment method:

    USE sql_invoicing;
    SELECT 
        p.date,
        p.invoice_id,
        p.amount,
        c.name,
        pm.name AS payment 
    FROM payments p
    JOIN clients c 
        ON p.client_id = c.client_id 
    JOIN payment_methods pm
        ON p.payment_method = pm.payment_method_id 
        
**18. Compound Join Conditions**

In all examples so far, we used a single column to uniquely identify the rows in a given table. There are times when we cannot use a single column to uniquely identify records in a given table. In these cases we use a combination of values in multiple columns to **uniquely identify each record**.

Here, we need multiple conditions to join two tables. 

In some tables we have a **composite primary key. A composite primary key contains more than one column**. When you have a table with a composite primary key you need to learn how to join that table with other tables: 

    USE sql_store;
    SELECT *
    FROM order_items oi
    JOIN order_item_notes oin
        ON oi.order_id = oin.order_id
        AND oi.product_id = oin.product_id

**19. Implicit Join Syntax**

We have the following basic JOIN (explicit join):

    SELECT *
    FROM orders o
    JOIN customers c
        ON o.customer_id = c.customer_id 
        
There is another way to write this query using implicit join syntax:

    SELECT *
    FROM orders o, customers c
    WHERE o.customer_id = c.customer_id 

Same as explicit join, we will get 10 records (we have 10 in each orders and customers tables).

Although MySQL supports this syntax I suggest you not to use this implicit join syntax b/c if you accidentally forget to type the WHERE clause you will get a cross join:

    SELECT *
    FROM orders o, customers c
    
In this case every record in the orders table is joined with every record in the customers table and we end up having 100 records! Later in the section we talk in more details about cross join. 

**Lesson ---->**  avoid using implicit join syntax and instead use the explicit join syntax! 

**20. Outer Joins**

With the (INNER) JOIN we only get the records meeting the condition we specified in JOIN (after ON) but what if we want to get other records, not meeting the condition, as well? **Use OUTER JOIN**

We have two types of (OUTER) JOIN:

+ LEFT, all the records in the LEFT table (the one comes first in front of FROM is returned whether or not the JOIN condition is True) 

+ RIGHT, all the records in the RIGHT table (the one comes in front of JOIN is returned whether or not the JOIN condition is True)

    USE sql_store;
    SELECT 
        p.product_id,
        p.name,
        oi.quantity
    FROM products p
    LEFT JOIN order_items oi 
        ON p.product_id = oi.product_id 
        
So whenever you see JOIN it is inner join and LEFT/RIGHT JOIN it is outer join. 
**As a best practice avoid RIGHT JOIN and always use LEFT JOIN instead!**

**21. Outer Joins between Multiple Tables**

**As a best practice avoid RIGHT JOIN and always use LEFT JOIN instead! It is easier to visualize your query.**

    USE sql_store;
    SELECT 
        o.order_id,
        o.order_date,
        c.first_name AS customer, 
        sh.name AS shipper,
        os.name AS status 
    FROM orders o
    JOIN customers c 
        ON o.customer_id = c.customer_id 
    LEFT JOIN shippers sh
        ON o.shipper_id = sh.shipper_id 
    JOIN order_statuses os
        ON o.status = os.order_status_id 

**22. Self Outer Joins**

    USE sql_hr;
    SELECT 
        e.employee_id,
        e.first_name AS employee
        m.first_name AS manager 
    FROM employees e
    LEFT JOIN employees m
        ON e.reports_to = m.employee_id 

**23. The Using Clause**

We can use USING clause with both INNER and OUTER JOIN. 

We use USING clause to make our query shorter and cleaner, we use it **whenever the column name in both tables are EXACTLY the same**, like

    USE sql_store;
    SELECT 
        o.order_id,
        c.first_name,
        sh.name AS shipper
    FROM orders o
    JOIN customers c
        ON o.customer_id = c.customer_id 
        
The simpler query using USING keyword is: 

    USE sql_store;
    SELECT 
        o.order_id,
        c.first_name,
        sh.name AS shipper
    FROM orders o
    JOIN customers c
        USING (customer_id)  
    LEFT JOIN shippers sh
        USING (shipper_id) 
        
Again remember, USING clause works **ONLY if the columns are of exactly the same name across multiple tables**.

What if we have multiple columns in our JOIN condition? If you remember we had the following before:

    SELECT *
    FROM order_items oi
    JOIN order_item_notes oin
        ON oi.order_id = oin.order_id
        AND oi.product_id = oin.product_id
        
The above JOIN condition is messy we can simplify it using USING keyword:

    SELECT *
    FROM order_items oi
    JOIN order_item_notes oin
        USINT (order_id, product_id) 
        
Exercise:

Write a query to select payments from the payments table and produce a report including date, client, amount, and name of the payment method:

    USE sql_invoicing;
    SELECT 
        p.date,
        c.name AS client,
        p.amount,
        pm.name AS payment_method 
    FROM clients c
    JOIN payments p USING (client_id)
    JOIN payment_methods pm
        ON p.payment_method = pm.payment_method_id -- here I cannot use USING clause b/c the name of the column in the p and pm tables are NOT the same! 

**24.Natural Joins**

In MySQL we also have another simpler way to join two tables, which is easier to code BUT **NOT RECOMMENDED!** Because it sometimes produces unexpected results. You will be presented here to make sure you understand it if you see it somewhere but don’t use it yourself. 

    USE sql_store;
    SELECT
        o.order_id,
        c.first_name
    FROM orders o
    NATURAL JOIN customers c
    
**With natural JOIN we don’t explicitly specify the column names. The database engine looks at these two tables and join them based on the common columns i.e., columns that have the same names that is the reason this query is shorter to write.**

**25. Cross Joins**

We use cross JOIN to join every record in the first table with every record in the second table. So that is why we don’t have a condition using ON keyword in the cross JOIN. 

    USE sql_store;
    SELECT 
        c.first_name AS customer,
        p.name AS product
    FROM customers c
    CROSS JOIN products p
    ORDER BY c.first_name 
    
What we have above is called the explicit syntax for CROSS JOIN. We also have implicit syntax which looks like the following: 
Instead of typing CROSS JOIN products p we type multiple tables in FROM clause like:

    USE sql_store;
    SELECT
        c.first_name AS customer,
        p.name AS product
    FROM customers c, products p
    ORDER BY c.first_name
    
MOSH personally prefers the explicit syntax b/c it is more clear. 

**26. Unions**

So far we learned that how to join columns from multiple tables. But in SQL we can also JOIN rows from multiple tables, which is extremely powerful. 

Using UNION operator we can combine records from multiple queries. 

    USE sql_store;
    SELECT 
        Order_id,
        Order_date,
        ‘Active’ AS status
    FROM orders o
    WHERE o.order_date >= “2019-01-01” -- here I hard coded the current year date, which does not deliver the desired results next year and so we will learn how to not hard code this later in the course. 
    UNION
    SELECT 
        ordare_id,
        order_date,
        ‘Archived’ AS status 
    FROM orders o
    WHERE o.order_date < “2019-01-01”
    
In the above both the queries are from the same table, **you also can query from different tables** and then using UNION operator you can combine the results into one result set. 

Like:

    SELECT first_name
    FROM customers
    UNION
    SELECT name
    FROM shippers 
    
The name of the column in the returned results is based on the first query like in the above the name of the column in the returned result is first_name. we can also have an alias like SELECT first_name AS full_name. 

Just remember the number of columns each query returns should be equal otherwise you will get an error. We get an error running the following b/c the first query returns 2 columns and the second one returns only one column!

    SELECT first_name, last_name 
    FROM customers
    UNION
    SELECT name
    FROM shippers 
    
Exercise:

    Points < 2000 >>> type would be Bronze 
    2000 <Points < 3000 >>> type would be Silver  
    Points > 3000 >>> type would be Gold 
    And also sort the results by the first name of the customers. 
 
My solution:

    USE sql_store;
    SELECT 
        customer_id,
        first_name,
        points,
        ‘Gold’ AS type
    FROM customers 
    WHERE points > 3000
    UNION
    SELECT 
        customer_id,
        first_name,
        points,
        ‘Silver’ AS type
    FROM customers
    WHERE points BETWEEN 2000 AND 3000
    UNION
    SELECT 
        customer_id,
        first_name,
        points,
        ‘Bronze’ AS type
    FROM customers
    WHERE points < 2000
    ORDER BY first_name 

#  Inserting, Updating, and Deleting Data

**27. Column attributes**

In MySQL workbench, I can open a table in a design mode through clicking on the tool sign beside table name and learn about column attributes. But in Pandas, I can get the column attributes using my following function:

    def column_attribute (database, table_name):

        con = connector(database)

        query = """
            SELECT column_name, data_type
            FROM information_schema.columns
            WHERE table_name = '{}'
        """.format(table_name)

        # read the query result into a DataFrame
        df = pd.read_sql(query, con)
        return df
    
    

**28. VARCHAR vs. CHAR**

VARCHAR and CHAR are both SQL data types used to store character strings, but they have some differences in terms of their storage and usage.

**CHAR is a fixed-length string data type**, which means that it always allocates a fixed amount of space for each value, regardless of whether the value needs it or not. For example, if you define a column as CHAR(10), each value in that column will take up exactly 10 bytes of storage, regardless of whether the actual value is shorter or longer than 10 characters. This makes CHAR more efficient for fixed-length data, such as postal codes or phone numbers.

**VARCHAR, on the other hand, is a variable-length string data type**. It allocates only as much space as the value needs, plus a small amount of overhead to store the length of the value. For example, if you define a column as VARCHAR(10), a value that is 5 characters long will take up only 5 bytes of storage, plus an additional byte to store the length of the value. This makes VARCHAR more efficient for variable-length data, such as names or addresses.

**In general, it is recommended to use VARCHAR for most string data types, unless you know that the data will always be fixed-length.** The reason is that **VARCHAR is more flexible and efficient in terms of storage space.** However, if you know that your data will always be fixed-length, **CHAR can be more efficient in terms of processing time, as it eliminates the need to calculate the length of each value during queries.**

Here is an example of how you can create a table with CHAR and VARCHAR columns in SQL:

sql code:


    CREATE TABLE users (
        id INT PRIMARY KEY,
        postal CHAR(50),
        email VARCHAR(100)
    );

In this example, the postal column is defined as a CHAR column with a fixed length of 50 characters, while the email column is defined as a VARCHAR column with a maximum length of 100 characters.

**29. Inserting a single row into a table**

    USE sql_store;
    INSER INTO customers
    VALUES (
        DEFAULT,
        ‘John’,
        ‘Smith’,
        ‘1990-01-01’,
        NULL, 
        ’address’,
        ‘city’,
        ‘CA,
        DEFAULT)
        
This way we can add a row to our table called customers. However, since we only provide values for some of the columns there is another way to write this statement:

In this way we can optionally supply the name of columns we want to supply values for, this way we don’t need to provide the default or null values as above: 

    INSERT INTO customers (
                    first_name,
                    last_name,
                    birth_date,
                    address,
                    city,
                    state)
    VALUES(
                    ‘John’,
                    ‘Smith’,
                    ‘1990-01-01’,
                    ’address’,
                    ‘city’,
                    ‘CA)
                    

This way we can also change the order of columns in the statement and so we don’t need to follow the columns order as in the table like:

    INSERT INTO customers (
                        last_name,
                        first_name,
                        birth_date,
                        address,
                        city,
                        state)
    VALUES(
                        ‘Smith,
                        ‘John,
                        ‘1990-01-01’,
                        ’address’,
                        ‘city’,
                        ‘CA)
                        
Be aware of opening a table in a design mode through clicking on the tool sign beside table name and learn about column attributes. Notice for the customer_id column the **AI (Auto increment)** is checked. Pay attention that here MySQL automatically generates the value of 11 for the customer_id value for the new row; it gets the value of the customer_id column for the last row and increments it by one and assigns it to the customer_id value for the new added row. 

To achieve the same using Pandas, I need:

    con = connector('sql_store')
    cursor = con.cursor()

    cursor.execute("""
        INSERT INTO shippers (name) 
        VALUES  ('shipper1')

    """)

    con.commit()


**30. multiline string**

The **"""** at the beginning and end of the query is called a multiline string, or a triple-quoted string. It is a **Python syntax feature that allows you to define a string that spans multiple lines of code.**

In this case, the triple-quoted string is used to define a multiline SQL query that spans multiple lines of code in the Python script. It makes the code more readable and easier to maintain, especially for longer SQL queries that might be harder to read if they were defined as a single line of code.

Note that triple-quoted strings are not required to execute SQL queries in Python, and you can define SQL queries as regular strings using single or double quotes. However, when defining long SQL queries, it can be useful to use triple-quoted strings to make the code more readable.

example:

    pd.read_sql("""
                SELECT *
                FROM customers

                """, con)
                
                
**31. Inserting Multiple Rows**

We just need to supply different rows’ values in separate parenthesis in front of VALUES like

    USE sql_store;
    INSERT INTO shippers (name)
    VALUES (‘shipper1’),
        (‘shipper2’),
        (‘shipper3’) 

**32. Inserting Hierarchical rows**

So far we learned how to enter data into a single table. Here we will learn **how to insert data into multiple tables**. 

We have a parent table and a child table like in our sql_store database we have orders table as parent table and order_items table as child table. So one row in orders table can have one or more children inside the order_items table. We want to insert an order and all its items, which requires us to insert data into multiple tables:

        USE sql_store;
        INSERT INTO orders (customer_id, order_date, status) -- order_id is auto increment of orders table and so it will be generated, I need to get it through executing LAST_INSERT_ID () function to use it to insert the child record into the child table i.e., order_items
        VALUES (1, ‘2019-01-02’, 1);
        INSERT INTO order_items 
        VALUES 
            (LAST_INSERT_ID(), 1, 1, 2.95),
            (LAST_INSERT_ID(), 2, 1, 3.95)
    
Some points:

In MySQL there are bunch of built-in functions. We can call/execute the function **LAST_INSERT_ID ()** to access the ID that MySQL generates when we insert a new row. We use LAST_INSERT_ID () to get the ID of the newly inserted record. So we can use that ID to insert the child record. 

note on running sql code in Pandas:

I needed to separate the two INSERT queries like in the following otherwise I got programmingerror:

    cursor = con.cursor()

    cursor.execute("""

    INSERT INTO orders (customer_id, order_date, status)
    VALUES (1, '2019-01-02', 1); 

    """)

    cursor.execute("""

    INSERT INTO order_items
    VALUES 
        (LAST_INSERT_ID(), 1, 1, 2)
    """)

    con.commit()
    
**33. Creating a Copy of a Table**

We will learn how to copy data from one table to another. 

Let’s say we want to make an exact copy of table orders, we can use the following:

CREATE TABLE name_of_the_new_table AS then we use SELECT statement to get everything from orders table:

    USE sql_store;
    CREATE TABLE orders_archived AS
    SELECT *
    FROM orders 
    
This way we can create an exact copy of orders table, but if we open the orders_archived table in design mode we see in this table **we don’t have a primary key and auto increment** for this table. So when creating a table using this technique MySQL ignores these attributes and that means if we want explicitly insert a record into the orders_archived we have to supply a value for order_id as well (order_id is the primary key and also marked as auto increment in the orders table). 

In the above query we refer to the SELECT statement as a **subquery**. **A subquery is a SELECT statement that is part of another SQL statement**. We can also use a subquery in an INSERT statement and that is a very powerful technique like:

**Side note**: To delete all the data in a table we can right click on the table name in the schemas and then click **Truncate Table**, this deletes all the data in the table. 

I can achieve this using Python as follows:

    con = connector('sql_store')
    
    cursor = con.cursor()
    
    cursor.execute("""

    TRUNCATE TABLE orders_archived

    """)

    con.commit()

    con.close()


How to copy a subset of a table into a new table? **using a subquery in an INSERT statement**, like in the following: 

Here we want to copy all the orders before 2019 into the table orders_archived:
We can use the SELECT statement as a subquery to the INSERT statement:

    USE sql_store;
    INSERT INTO orders_archived 
    SELECT *
    FROM orders
    WHERE order_date <  ‘2019-01-01’
    
Exercise:

    USE sql_invoicing;
    CREATE TABLE invoice_archived AS 
    SELECT 
        i.invoice_id,
        i.number,
        c.name AS client,
        i.invoice_total, 
        i.payment_total,
        i.invoice_date,
        i.payment_date,
        i.due_date
    FROM clients c
    JOIN invoices i USING (client_id) 
    WHERE payment_date IS NOT NULL 

Note: 

Because the columns invoice_date, payment_date and due_date only exist in invoices table we don’t need to prefix their names in the SELECT statement but MOSH prefers to do so to make his query more clear.

**34. Updating a Single Row**

here